Previously in assignments 2 and 3, we trained fully connected networks to classify notMNIST characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
+ convolutions need the image data formatted as a cube (width by height by #channels)
+ labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's `build a small convolutional neural network` with:
+ `Two convolutional layers`.
+ `Followed by one fully connected layer`.

`Convolutional networks` are more `expensive` computationally, so `we'll limit its depth` and `number of fully connected nodes`.

Output size is calculated as follows:

    O = ((I - K + 2P)/S) + 1,       for every I,K,P,S.

where 
    + O: output size
    + I: input feature map size
    + K: kernel size
    + P: padding ("SAME":-1 | "VALID":0)
    + S: strides

* Note: zero padding, non-unit strides (relationship 6)



In [20]:
output_size_1 = ((28.0 - 5.0 + 2.0*(-1))/2.0) + 1.0
print(np.ceil(output_size_1))
output_size_2 = ((output_size_1 - 5.0 + 2.0*(-1))/2.0) + 1.0
print(np.ceil(output_size_2))

12.0
4.0


In [11]:
batch_size = 16
patch_size = 5 ## kernel size
depth = 16 
num_hidden = 64 ## hidden neurons

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) ## stddev=0.1 to avoid dead neurons.
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    ## nn.conv2d parameters explanation: https://stackoverflow.com/a/34698115/2881205
    ## strides: [1,2,2,1] = [images, height, width, channels]
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 1001 ## 89.8%
# num_steps = 10001 ## 94.7%
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.017948
Minibatch accuracy: 18.8%
Validation accuracy: 13.3%
Minibatch loss at step 50: 1.419065
Minibatch accuracy: 62.5%
Validation accuracy: 52.7%
Minibatch loss at step 100: 0.635819
Minibatch accuracy: 81.2%
Validation accuracy: 70.5%
Minibatch loss at step 150: 1.130260
Minibatch accuracy: 50.0%
Validation accuracy: 73.8%
Minibatch loss at step 200: 0.470542
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 250: 0.808304
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 300: 1.007090
Minibatch accuracy: 68.8%
Validation accuracy: 77.7%
Minibatch loss at step 350: 1.133507
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 400: 0.440134
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.265464
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.609530
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
M

By increasing the number of steps 1001 (with step%50) to 10001 (with step%500), the test accuracy is improved from 89.6% to 94.7%, respectively. 

# Problem 1
The convolutional model above uses convolutions with `stride 2` to reduce the dimensionality. Replace the strides by a `max pooling operation` (`nn.max_pool()`) of `stride 2` and `kernel size 2`.

In [24]:
batch_size = 16
patch_size = 5  
depth = 16 # number of output channels
num_hidden = 64


graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    ## Convolutional layer 1: Input channel=1; Output channels=16 (depth).
    weights_1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    biases_1 = tf.Variable(tf.zeros([depth]))
    
    ## Convolutional layer 2: Input channels=16 (depth); Output channels=16 (depth).
    weights_2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    ## Fully-connected layer
    weights_3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    biases_3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    ## Softmax
    weights_4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    biases_4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Max Pooling 
    ## args: data, kernel_size
    def max_pool_2d(data, k): 
        return tf.nn.max_pool(data, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
        
    # ConvNet Model
    def model(data):
        # convolutional layer 1
        conv = tf.nn.conv2d(data, weights_1, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + biases_1)
        # max pooling operation
        pooling = max_pool_2d(hidden, 2)
        
        # convolutional layer 2
        conv = tf.nn.conv2d(pooling, weights_2, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + biases_2)
        # max pooling operation
        pooling = max_pool_2d(hidden, 2)
        
        # combine into the output size
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
        
        # Fully-connected layer
        hidden = tf.nn.relu(tf.matmul(reshape, weights_3) + biases_3)
        return tf.matmul(hidden, weights_4) + biases_4
    
    # Training computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer
    ## learning_rate = 0.05
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [25]:
# num_steps = 1001 ## with steps%50: 90.3%
num_steps = 30001 ## with steps%5000: 95.6%

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.313919
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 5000: 0.907293
Minibatch accuracy: 68.8%
Validation accuracy: 86.9%
Minibatch loss at step 10000: 0.232737
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 15000: 0.256875
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 20000: 0.385056
Minibatch accuracy: 87.5%
Validation accuracy: 89.4%
Minibatch loss at step 25000: 0.049083
Minibatch accuracy: 100.0%
Validation accuracy: 90.0%
Minibatch loss at step 30000: 0.345991
Minibatch accuracy: 87.5%
Validation accuracy: 90.0%
Test accuracy: 95.6%


# Problem 2
Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

* Adding in the fully-connected layer:
    + Regularization
    + Learning rate decay
    + Dropout (at training, not at evaluation).
    
   

In [48]:
batch_size = 16
patch_size = 5  
depth = 16 # number of output channels
num_hidden = 64
beta = 0.001

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    ## Convolutional layer 1: Input channel=1; Output channels=16 (depth).
    weights_1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    biases_1 = tf.Variable(tf.zeros([depth]))
    
    ## Convolutional layer 2: Input channels=16 (depth); Output channels=16 (depth).
    weights_2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    ## Fully-connected layer
    weights_3 = tf.Variable(tf.truncated_normal([image_size//4 * image_size//4 * depth, num_hidden], stddev=0.1))
    biases_3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    ## Softmax
    weights_4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    biases_4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Max Pooling 
    ## args: data, kernel_size
    def max_pool_2d(data, k): 
        return tf.nn.max_pool(data, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
    
    # ConvNet Model
    def model(data):
        # convolutional layer 1
        conv_1 = tf.nn.conv2d(data, weights_1, [1, 1, 1, 1], padding='SAME')
        hidden_1 = tf.nn.relu(conv_1 + biases_1)
        # max pooling operation
        pooling_1 = max_pool_2d(hidden_1, 2)
        
        # convolutional layer 2
        conv_2 = tf.nn.conv2d(pooling_1, weights_2, [1, 1, 1, 1], padding='SAME')
        hidden_2 = tf.nn.relu(conv_2 + biases_2)
        pooling = max_pool_2d(hidden_2, 2)
        
        # combine into the output size
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
        
        # Fully-connected layer
        hidden = tf.nn.relu(tf.matmul(reshape, weights_3) + biases_3)
        # dropout
        keep_prob = 0.5
        hidden_dropout = tf.nn.dropout(hidden, keep_prob)
        return tf.matmul(hidden_dropout, weights_4) + biases_4
    
    # Training computation
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # with L2 regularization
    # newloss = loss + beta*regularization
    regularization = tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4)
    loss = tf.reduce_mean(loss + beta*regularization)
    
    # Optimizer
    ## Use Learning Rate Decaying.
    global_step = tf.Variable(0) ## count the number of steps taken
    starter_learning_rate = 0.05
    number_decay_steps = 100000
    base_of = 0.96

    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, number_decay_steps, base_of, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [50]:
num_steps = 30001 ## with steps%5000: 93.7%

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.182820
Minibatch accuracy: 6.2%
Validation accuracy: 10.3%
Minibatch loss at step 5000: 1.161717
Minibatch accuracy: 62.5%
Validation accuracy: 84.1%
Minibatch loss at step 10000: 0.425975
Minibatch accuracy: 93.8%
Validation accuracy: 85.8%
Minibatch loss at step 15000: 0.434305
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 20000: 0.640733
Minibatch accuracy: 81.2%
Validation accuracy: 87.2%
Minibatch loss at step 25000: 0.155574
Minibatch accuracy: 100.0%
Validation accuracy: 87.3%
Minibatch loss at step 30000: 0.352715
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Test accuracy: 93.7%
